# NHS KPI Categorisation & Simulation Project


### KPI Performance Category Definitions

| Category  | Meaning                                                                 | Example Criteria                         |
|-----------|-------------------------------------------------------------------------|------------------------------------------|
| **Gold**  | Highest performance; exceeds expected target                           | Compliance ≥ 95%                         |
| **Silver**| Acceptable performance; meets standard but not optimal                 | Compliance between 90–94%                |
| **Bronze**| Below expectation; needs improvement                                    | Compliance between 85–89%                |
| **White** | Unclassified or undefined; outside all thresholds or missing/incomplete | No value, out-of-band, or ambiguous data |
| **Manual**| Not evaluated automatically; requires human review                     | Text thresholds like “More than comparator” |


In [ ]:
# Import necessary libraries
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Suppress warning messages for cleaner output
warnings.filterwarnings("ignore")

### Step 1: Load KPI Data
Begin by loading the core dataset containing ward-level KPI values. Each row represents a monthly measurement for a specific KPI in a given ward.  
The goal is to later classify these values into performance categories (Gold, Silver, etc.) based on standard thresholds.

**Columns overview**:
- `Ward`: Name of the unit or ward  
- `KPI_Name`: The specific key performance indicator  
- `Year` and `Month`: Time period of the KPI recording  
- `KPI_Value`: The recorded value, which may be a percenumber or text or score


In [ ]:
# Load KPI data
kpi_data = pd.read_csv("KPI_Data_2024.csv")
print("KPI Data Sample:")
display(kpi_data.head())

### Step 2: Load KPI Thresholds

Now load a reference table that defines performance thresholds for each KPI.  
These thresholds determine what constitutes "Gold", "Silver", "Bronze", etc., and the expected direction (high/low). Some KPIs are flagged as "manual" and will be excluded from logic-based classification.

**Columns include**:
- `KPI_Name`: Matching key with the KPI data
- `Direction`: Whether higher or lower values are better
-  Thresholds for each category band
- `Manual_Flag`: Indicates if manual review is needed

In [ ]:
# Load Thresholds data
thresholds = pd.read_csv("KPI_Threshold_Lookup_Table.csv")

print("\nThreshold Table Sample:")
display(thresholds.head())

### Step 3: Inspect KPI Data Types and Value Format

Before applying classification logic, it's important to ensure the `KPI_Value` column contains valid numeric entries.  
Threshold comparisons rely on numerical values, and mixed or invalid formats may lead to incorrect categorisation.

- The data types of each column are reviewed to confirm structural consistency.
- Any non-numeric entries in `KPI_Value` are identified and displayed for further attetion.
view.


In [ ]:
# Check data types of KPI data
display(kpi_data.dtypes)

print("-----------------------------")
# Identify non-numeric entries in KPI_Value column
non_numeric_rows = kpi_data[~kpi_data['KPI_Value'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notnull()]

print(non_numeric_rows)

### Step 4: Confirm KPI Name Consistency and Merge Datasets

Before merging the KPI data with the threshold lookup table, it's useful to confirm that all `KPI_Name` values match correctly between both sources.  
This avoids join issues and ensures each KPI receives the appropriate threshold logic.

- A left join is used to retain all rows from the main dataset.
- The merged dataset includes additional columns such as `Direction`, thresholds for each category, and the manualflag.


In [ ]:
# Display unique KPI names from both datasets to verify alignment
print("Unique KPI Names in KPI Data:")
print(kpi_data['KPI_Name'].unique())

print("\nUnique KPI Names in Threshold Table:")
print(thresholds['KPI_Name'].unique())

# Merge KPI data with thresholds using KPI name
merged_data = pd.merge(kpi_data, thresholds, on='KPI_Name', how='left')

print("\nMerged Data Sample:")
display(merged_data.head())

### Step 5: Validate Merge – Check for Missing Thresholds

After merging, it's important to confirm that all `KPI_Name` values were successfully matched with a threshold definition.  
This check filters for any rows where `Direction` is missing, indicating an unmatched or undefined KPI.

An empty result confirms that all KPIs in the dataset have corresponding threshold logic availble.


In [ ]:
# Identify any KPI records that did not match a threshold (missing direction)
missing_thresholds = merged_data[merged_data['Direction'].isnull()]

print("\nRows with missing threshold match:")
print(missing_thresholds)

### Step 6: Define Categorisation Logic

This function assigns each KPI record to a performance category based on predefined thresholds.

- If a KPI is flagged as "Manual", it is assigned that category directly.
- If `Direction` is "high", higher values are better.
- If `Direction` is "low", lower values are better.
- Values falling outside all threshold ranges are marked as "White" (unclassfied).


In [ ]:
def assign_category(row):
    """
    Assigns a KPI to Gold, Silver, Bronze, White, or Manual based on its value, direction, and thresholds.
    """
    if row['Manual_Flag'] == "Yes":
        return "Manual"

    value = row['KPI_Value']
    if pd.isna(value):
        return "White"

    direction = row['Direction']

    if direction == "high":
        if value >= row['Gold_Threshold']:
            return "Gold"
        elif row['Silver_Min'] <= value <= row['Silver_Max']:
            return "Silver"
        elif row['Bronze_Min'] <= value <= row['Bronze_Max']:
            return "Bronze"
        else:
            return "White"

    elif direction == "low":
        if value <= row['Gold_Threshold']:
            return "Gold"
        elif row['Silver_Max'] <= value <= row['Silver_Min']:
            return "Silver"
        elif row['Bronze_Max'] <= value <= row['Bronze_Min']:
            return "Bronze"
        else:
            return "White"

    return "White"  # Fallback

### Step 7: Apply Categorisation Logic to KPI Records

The `assign_category` function is applied row-wise to the merged dataset to assign a performance category to each KPI entry.  
This results in a new column called `Assigned_Category`.

The table below shows the first 10 categorised rows for validation.

In [ ]:
# Apply categorisation logic to each row
merged_data['Assigned_Category'] = merged_data.apply(assign_category, axis=1)

# Preview categorised results
print("Categorised Sample:")
display(merged_data[['Ward', 'KPI_Name', 'Month', 'KPI_Value', 'Assigned_Category']].head(10))

## _Categorised Output for Dashboarding_ ##

In [ ]:
# Export categorised KPI data to Excel
categorised_output = merged_data[['Ward', 'KPI_Name', 'Month', 'KPI_Value', 'Assigned_Category']]
categorised_output.to_excel('Categorised_KPI_Results.xlsx', index=False)

print("Export complete. File saved as 'Categorised_KPI_Results.xlsx'")

### Step 8: Generate Category Counts by Ward and Month

This summary table shows the number of KPIs assigned to each category (Gold, Silver, Bronze, etc.) for every ward and month.  
It provides a quick overview of overall performance distribution at the ward levl.


In [ ]:
# Count number of KPIs in each category for each Ward-Month combination
category_counts = merged_data.groupby(['Ward', 'Month', 'Assigned_Category']).size().unstack(fill_value=0).reset_index()

print("Category counts per Ward + Month:")
display(category_counts.head())

### Step 9: Calculate Category Distribution as Percentages

To assess overall annual performance, the total count of each KPI category is summed for each ward across all months.  
These totals are then converted into percentages to represent the distribution of KPI outcomes per ward.

This helps quickly identify which wards perform consistently well (e.g. higher % Gold) or may need support (e.g. higher % White or Manual).

In [ ]:
# Step 1: Sum all category counts by ward (across all months)
ward_summary = category_counts.groupby('Ward')[['Gold', 'Silver', 'Bronze', 'Manual', 'White']].sum()

# Step 2: Convert to percentage distribution per ward
ward_percent = ward_summary.div(ward_summary.sum(axis=1), axis=0) * 100
ward_percent = ward_percent.round(0).astype(int)

# Preview the percentage table
print("Category % by Ward (Annual Summary):")
display(ward_percent.head())

### Step 10: Add Total KPI Count and Gold Percentage per Ward-Month

This step calculates:
- The total number of KPIs recorded per ward per month (`Total_KPIs`)
- The percentage of those KPIs that achieved a "Gold" rating (`Gold_Percentage`)

This provides a quick metric to assess overall monthly performance strength at the ward evel.


In [ ]:
# Add total KPI count across categories for each row
category_counts['Total_KPIs'] = category_counts[['Gold', 'Silver', 'Bronze', 'Manual', 'White']].sum(axis=1)

# Calculate the percentage of Gold-rated KPIs
category_counts['Gold_Percentage'] = (category_counts['Gold'] / category_counts['Total_KPIs']) * 100

# Preview the updated summary
print("\nAdded % Gold column:")
display(category_counts[['Ward', 'Month', 'Gold', 'Total_KPIs', 'Gold_Percentage']].head())

### Step 11: Assign Performance Ratings Based on Gold Percentage

A final performance rating is assigned to each ward-month based on the percentage of KPIs rated as **"Gold"**.

This provides a quick, qualitative summary of monthly performance levels:

- **Excellent** – ≥ 90% of KPIs rated Gold  
- **Good** – 70% to 89% rated Gold  
- **Needs Improvement** – 50% to 69% rated Gold  
- **Needs Attention** – Less than 50% rd Gold



tention


In [ ]:
# Step 3: Define performance based on % Gold
def overall_performance(pct):
    if pct >= 90:
        return "Excellent"
    elif pct >= 70:
        return "Good"
    elif pct >= 50:
        return "Needs Improvement"
    else:
        return "Needs attention"

category_counts['Overall_Performance'] = category_counts['Gold_Percentage'].apply(overall_performance)

print("\nFinal Summary Table:")
display(category_counts[['Ward', 'Month', 'Gold_Percentage', 'Overall_Performance']].head())

##  Simulation 1: Stricter Gold Threshold for High-Value KPIs

This simulation explores how performance categorisation would change if the Gold standard were raised from its original value to **97** for KPIs where higher values are better.

**Purpose**:
- Assess the impact of stricter policy on reported performance
- Identify KPIs or months that would lose Gold status under tougher criteria
- Support decision-makers in understanding the sensitivity of current performance levels

Only KPIs with `Direction = "high"` are affected  by thi change.


In [ ]:
# Create a copy of the merged dataset for simulation
simulated_data = merged_data.copy()

# Tighten the Gold threshold for KPIs where higher values are better
simulated_data.loc[simulated_data['Direction'] == 'high', 'Gold_Threshold'] = 97

# Re-apply categorisation logic with new threshold
simulated_data['Simulated_Category'] = simulated_data.apply(assign_category, axis=1)

# Compare original vs simulated categories
comparison = simulated_data[['Ward', 'KPI_Name', 'Month', 'KPI_Value', 'Assigned_Category', 'Simulated_Category']]
comparison['Changed'] = comparison['Assigned_Category'] != comparison['Simulated_Category']

# Display sample results
print("Sample Simulation Comparison:")
display(comparison.head(10))

In [ ]:
# Count how many records changed due to the tighter threshold
changed_count = comparison['Changed'].sum()
total_rows = comparison.shape[0]
change_percentage = (changed_count / total_rows) * 100

print(f"\nTotal KPIs affected by stricter threshold: {changed_count} out of {total_rows} ({change_percentage:.1f}%)")

In [ ]:
plt.pie(
    [comparison['Changed'].sum(), (~comparison['Changed']).sum()],
    labels=['Changed', 'Unchanged'],
    colors=['#33a02c', '#b2df8a'],
    autopct='%1.1f%%',
    startangle=90,
    explode=[0.1, 0],
    textprops={'fontsize': 12}
)
plt.title("Impact of Stricter Gold Threshold", fontsize=12, weight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Count category distributions before and after
before_counts = merged_data['Assigned_Category'].value_counts().sort_index()
after_counts = simulated_data['Simulated_Category'].value_counts().sort_index()

# Ensure fixed category order (optional)
ordered_categories = ['Gold', 'Silver', 'Bronze', 'Manual', 'White']
before_counts = before_counts.reindex(ordered_categories)
after_counts = after_counts.reindex(ordered_categories)

# Combine into a DataFrame
threshold_shift = pd.DataFrame({'Before': before_counts, 'After': after_counts})

# Plot grouped bar chart
ax = threshold_shift.plot(
    kind='bar',
    figsize=(7, 4),
    color=['skyblue', 'steelblue'],
    edgecolor='lightblue'
)

# Add value labels
for bars in ax.containers:
    ax.bar_label(bars, fmt='%.0f', label_type='edge', fontsize=8)

# Styling
plt.title("Category Distribution Before vs After Gold Threshold Tightening", fontsize=12, weight='bold')
plt.xlabel("Performance Category")
plt.ylabel("Number of KPI Results")
plt.xticks(rotation=0)


plt.legend(title="Simulation Stage", loc='upper left', bbox_to_anchor=(0, 1))

# Clean up border
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

plt.tight_layout()
plt.show()

## Simulation 2: Direction-Aware Impact of a 3-Point Performance Drop

This simulation models a realistic decline in performance due to operational pressures such as staffing shortages, seasonal strain, or unexpected disruptions.

**Objective**:  
To assess how a moderate, direction-sensitive performance shift would affect KPI category assignments.

**Logic Applied**:
- KPIs where **higher values are better** (e.g., compliance rates): reduced by 3 points  
- KPIs where **lower values are better** (e.g., incident rates): increased by 3 points  
- KPIs marked for **manual review** were excluded from the simulation

This direction-aware adjustment helps identify:
- Which KPIs and wards are most vulnerable to slight underperformance
- Risk of losing Gold status under operational strain
- Overall fragility of current high-category performance

The results compare original category assignments to those recalculated after the simulatd value change.



In [ ]:
# Step 1: Ensure Assigned_Category is correctly populated
merged_data['Assigned_Category'] = merged_data.apply(assign_category, axis=1)

# Step 2: Create a fresh copy of the dataset for simulation
value_drop_sim = merged_data.copy()

# Step 3: Apply a direction-aware 3-point simulation
# For KPIs where higher is better: subtract 3 points
# For KPIs where lower is better: add 3 points (to simulate decline)
value_drop_sim['KPI_Value'] = value_drop_sim.apply(
    lambda row: (
        row['KPI_Value'] - 3 if row['Direction'] == 'high'
        else row['KPI_Value'] + 3 if row['Direction'] == 'low'
        else row['KPI_Value']
    ),
    axis=1
)

# Step 4: Re-apply categorisation logic on the simulated values
value_drop_sim['Drop_Category'] = value_drop_sim.apply(assign_category, axis=1)

# Step 5: Merge original and simulated data for comparison
drop_comparison = pd.merge(
    merged_data[['Ward', 'KPI_Name', 'Month', 'KPI_Value', 'Assigned_Category', 'Direction']],
    value_drop_sim[['Ward', 'KPI_Name', 'Month', 'KPI_Value', 'Drop_Category']],
    on=['Ward', 'KPI_Name', 'Month'],
    suffixes=('_Original', '_Simulated')
)

# Step 6: Identify rows where category has changed
drop_comparison['Changed'] = drop_comparison['Assigned_Category'] != drop_comparison['Drop_Category']

# Step 7: Filter changes by direction
low_direction_changed = drop_comparison[
    (drop_comparison['Direction'] == 'low') & (drop_comparison['Changed'])
]

high_direction_changed = drop_comparison[
    (drop_comparison['Direction'] == 'high') & (drop_comparison['Changed'])
]

In [ ]:
# View where low-direction KPIs had category changes
low_direction_changed = drop_comparison[
    (drop_comparison['Direction'] == 'low') &
    (drop_comparison['Changed'] == True)
]
print("\nLow-direction KPIs with changed categories post-drop (Sample):")
display(low_direction_changed.head())

# View where high-direction KPIs had category changes
high_direction_changed = drop_comparison[
    (drop_comparison['Direction'] == 'high') &
    (drop_comparison['Changed'] == True)
]
print("\nHigh-direction KPIs with changed categories post-drop (Sample):")
display(high_direction_changed.head())

## Summary of Simulation 2 Impact

The following outputs provide insights into the effects of a direction-aware 3-point performance change on KPI category classifications:

- **Table 1**: Shows the number of KPI category shifts for each Ward and KPI. This helps highlight indicators that are highly sensitive to performance drops and may signal vulnerable areas.
  
- **Pie Chart**: Summarises the overall proportion of KPI results that changed versus those that remained stable. This gives a quick sense of how performance categories are impacted under minor adverse shlience.
ty.


In [ ]:
# Count how many rows changed per Ward and KPI
change_summary = drop_comparison.groupby(['Ward', 'KPI_Name'])['Changed'].sum().reset_index()
change_summary = change_summary.rename(columns={'Changed': 'Count_Changed'})

print("Table1:Summary of KPI category changes per Ward:")
display(change_summary)

In [ ]:
# Group and count KPI category changes
top_changes = drop_comparison.groupby('KPI_Name')['Changed'].sum().reset_index()
top_changes.columns = ['KPI_Name', 'Count_Changed']

# Filter out KPIs with zero changes and sort descending
top_changes_filtered = top_changes[top_changes['Count_Changed'] > 0]
top_changes_filtered = top_changes_filtered.sort_values('Count_Changed', ascending=True)  # for horizontal barh chart, lowest at bottom

# Create green gradient color palette
colors = sns.color_palette("Greens", n_colors=len(top_changes_filtered))

# Plot
plt.figure(figsize=(7, 3))
bars = plt.barh(
    top_changes_filtered['KPI_Name'],
    top_changes_filtered['Count_Changed'],
    color=colors,
    edgecolor='black'
)

# Add value labels
for bar in bars:
    width = bar.get_width()
    plt.text(width + 1, bar.get_y() + bar.get_height() / 2, f'{int(width)}', va='center', fontsize=9)

# Styling
plt.title('KPIs Most Affected by 3-Point Value Drop', fontsize=13, weight='bold')
plt.xlabel('Number of KPI Category Changes')
plt.ylabel('KPIs')
plt.grid(axis='x', linestyle='--', alpha=0.6)
for spine in ['top', 'right']:
    plt.gca().spines[spine].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Summary of total changes caused by performance drop
drop_impact = drop_comparison['Changed'].sum()
total_records = drop_comparison.shape[0]
drop_rate = (drop_impact / total_records) * 100

print(f"\nTotal KPIs affected by 3-point drop: {drop_impact} out of {total_records} ({drop_rate:.1f}%)")

In [ ]:
# Count Changed vs Unchanged rows
change_counts = drop_comparison['Changed'].value_counts()
labels = ['Unchanged', 'Changed']
colors = ['#b2df8a', '#33a02c']  # Green tones

# Plot pie chart
plt.figure(figsize=(4, 4))
plt.pie(
    change_counts,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    textprops={'fontsize': 10}
)

plt.title('Overall Impact of 3-Point Value Drop', fontsize=10, weight='bold')
plt.tight_layout()
plt.show()

## Summary ##
This project aimed to resolve inconsistencies in ward-level KPI reporting by developing an automated, logic-driven categorisation and simulation tool. By integrating structured threshold logic, direction-aware value interpretation, and performance shift simulations, the solution standardises evaluation across wards and enhances audit transparency.

Through this implementation, NHS teams can:

-Reduce manual data handling time

-Identify performance risk areas more reliably

-Support evidence-based decision making for accreditation and improvement.

This notebook serves as a foundation for reproducible KPI analysis, adaptable to future changes in reporting standards or policy scenarios.